# SageMaker K-Means Clustering Exercise

This notebook demonstrates Amazon SageMaker's **K-Means** algorithm for unsupervised clustering.

## What You'll Learn
1. How to prepare data for clustering
2. How to configure and understand K-Means hyperparameters
3. How to train a K-Means model
4. How to interpret cluster assignments and evaluate clustering quality

## What is K-Means?

K-Means is an **unsupervised** algorithm that groups data points into k clusters by:
1. Initializing k cluster centroids (randomly or kmeans++)
2. Assigning each point to nearest centroid
3. Updating centroids to cluster means
4. Repeating until convergence

**SageMaker's Implementation:**
- Web-scale algorithm (handles billions of points)
- Single-pass streaming for efficiency
- GPU acceleration available
- Mini-batch variant for speed

## Use Cases

| Application | Description |
|-------------|-------------|
| Customer segmentation | Group customers by behavior |
| Anomaly detection | Find outliers far from centroids |
| Image compression | Reduce color palette |
| Document clustering | Group similar documents |
| Feature engineering | Create cluster membership features |

---

## ⚠️ Training Cost Information

<div style="background-color: #010302ff; border: 1px solid #28a745; border-radius: 5px; padding: 15px; margin: 10px 0;">

### K-Means Supports CPU and GPU

K-Means is computationally efficient. GPU helps with very large datasets.

| Instance Type | Type | On-Demand Price* | Best For |
|---------------|------|------------------|----------|
| ml.m5.large | CPU | ~$0.13/hour | Small datasets (<1M samples) |
| ml.c5.xlarge | CPU | ~$0.24/hour | Medium datasets |
| ml.g4dn.xlarge | GPU | ~$0.74/hour | Large datasets (>1M samples) |
| ml.p3.2xlarge | GPU | ~$3.83/hour | Very large datasets |

*Prices are approximate for us-west-2.

### Cost Estimation
- **Training**: Very fast! Usually 2-5 minutes (~$0.01-0.03)
- **Inference endpoint**: ~$0.13/hour for ml.m5.large
- K-Means is one of the most cost-effective SageMaker algorithms

### Key Advantage
SageMaker K-Means uses streaming single-pass algorithm - it can handle datasets that don't fit in memory!

</div>

## Step 1: Setup and Imports

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score

# Load environment variables from .env file
load_dotenv()

# Configure AWS session from environment variables
aws_profile = os.getenv('AWS_PROFILE')
aws_region = os.getenv('AWS_REGION', 'us-west-2')
sagemaker_role = os.getenv('SAGEMAKER_ROLE_ARN')

if aws_profile:
    boto3.setup_default_session(profile_name=aws_profile, region_name=aws_region)
else:
    boto3.setup_default_session(region_name=aws_region)

# SageMaker session and role
sagemaker_session = sagemaker.Session()

if sagemaker_role:
    role = sagemaker_role
else:
    role = get_execution_role()

region = sagemaker_session.boto_region_name

print(f"AWS Profile: {aws_profile or 'default'}")
print(f"SageMaker Role: {role}")
print(f"Region: {region}")
print(f"SageMaker SDK Version: {sagemaker.__version__}")

In [ ]:
# Configuration
BUCKET_NAME = sagemaker_session.default_bucket()
PREFIX = "kmeans"

# Dataset parameters
NUM_SAMPLES = 5000
NUM_FEATURES = 10
NUM_CLUSTERS = 5
RANDOM_STATE = 42

print(f"S3 Bucket: {BUCKET_NAME}")
print(f"S3 Prefix: {PREFIX}")

## Step 2: Generate Synthetic Clustering Data

In [ ]:
# Generate clustered data
X, y_true = make_blobs(
    n_samples=NUM_SAMPLES,
    n_features=NUM_FEATURES,
    centers=NUM_CLUSTERS,
    cluster_std=1.5,
    random_state=RANDOM_STATE
)

X = X.astype(np.float32)

print(f"Data shape: {X.shape}")
print(f"True clusters: {NUM_CLUSTERS}")
print(f"\nTrue cluster distribution:")
unique, counts = np.unique(y_true, return_counts=True)
for c, n in zip(unique, counts):
    print(f"  Cluster {c}: {n}")

In [ ]:
# Visualize first two dimensions
fig, ax = plt.subplots(figsize=(10, 7))

scatter = ax.scatter(X[:, 0], X[:, 1], c=y_true, cmap='tab10', alpha=0.6)
ax.set_xlabel('Feature 0')
ax.set_ylabel('Feature 1')
ax.set_title('True Clusters (First 2 Features)')
plt.colorbar(scatter, ax=ax, label='Cluster')
plt.show()

## Step 3: Prepare Data for K-Means

K-Means expects features only (no labels) in CSV or RecordIO format.

In [ ]:
# Save as CSV (no header, features only)
os.makedirs('data/kmeans', exist_ok=True)

np.savetxt('data/kmeans/train.csv', X, delimiter=',')

print(f"Saved: data/kmeans/train.csv ({os.path.getsize('data/kmeans/train.csv') / 1024:.1f} KB)")

# Preview
print("\nFile preview (first 3 lines):")
with open('data/kmeans/train.csv', 'r') as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(f"  {line.strip()[:80]}...")

In [ ]:
# Upload to S3
s3_client = boto3.client('s3')

train_s3_key = f"{PREFIX}/train/train.csv"
s3_client.upload_file('data/kmeans/train.csv', BUCKET_NAME, train_s3_key)

train_uri = f"s3://{BUCKET_NAME}/{PREFIX}/train"
print(f"Data uploaded to: {train_uri}")

## Step 4: Train K-Means Model

### Understanding K-Means Hyperparameters

| Parameter | Description | Default | Recommendation |
|-----------|-------------|---------|----------------|
| `k` | Number of clusters | Required | Use elbow/silhouette method |
| `feature_dim` | Number of features | Required | Must match data |
| `mini_batch_size` | Batch size | 5000 | 1000-10000 |
| `epochs` | Training passes | 1 | 1-10 |
| `init_method` | Initialization | random | `random` or `kmeans++` |
| `extra_center_factor` | Extra centers during training | 1 | 1-10 for stability |
| `eval_metrics` | Metrics to compute | None | `msd`, `ssd` |
| `half_life_time_size` | For streaming (decay factor) | 0 | For concept drift |
| `local_lloyd_max_iter` | Max Lloyd iterations | 300 | Usually sufficient |
| `local_lloyd_tol` | Convergence tolerance | 0.0001 | Lower = more precision |
| `local_lloyd_init_method` | Local initialization | kmeans++ | kmeans++, random |

### Key Hyperparameter Details

**k (Number of Clusters)**
- Most critical parameter
- Too few → heterogeneous clusters
- Too many → fragmented, meaningless clusters
- Use **Elbow Method** or **Silhouette Score** to determine optimal k
- Start with estimated number of natural groups

**init_method**
- `random`: Random initialization (faster, less stable)
- `kmeans++`: Smart initialization (more stable, slightly slower)
- **Recommendation**: Always use `kmeans++` unless speed is critical

**extra_center_factor**
- Creates k × extra_center_factor centers during training
- Reduces to k at the end
- Higher values = more stable results, longer training
- Use 2-5 for better quality

**mini_batch_size**
- Larger = faster training, potentially less accurate
- Smaller = slower, potentially better clusters
- SageMaker streams data, so batch size affects memory, not data size

### Evaluation Metrics

| Metric | Description | Good Values |
|--------|-------------|-------------|
| `msd` | Mean Squared Distance to centroid | Lower is better |
| `ssd` | Sum of Squared Distances (Inertia) | Lower is better |
| Silhouette Score | Cluster separation quality | Closer to 1 is better |

### CloudWatch Training Metrics

| Metric | Description |
|--------|-------------|
| `train:msd` | Training mean squared distance |
| `test:msd` | Test mean squared distance |

In [ ]:
# Get K-Means container image
kmeans_image = retrieve(
    framework='kmeans',
    region=region,
    version='1'
)

print(f"K-Means Image URI: {kmeans_image}")

In [ ]:
# Create K-Means estimator
kmeans_estimator = Estimator(
    image_uri=kmeans_image,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{BUCKET_NAME}/{PREFIX}/output',
    sagemaker_session=sagemaker_session,
    base_job_name='kmeans'
)

In [ ]:
# Set hyperparameters
hyperparameters = {
    "k": NUM_CLUSTERS,
    "feature_dim": NUM_FEATURES,
    "mini_batch_size": 500,
    "epochs": 10,
    "init_method": "random",
    "extra_center_factor": 2,
}

kmeans_estimator.set_hyperparameters(**hyperparameters)

print("K-Means hyperparameters:")
for k, v in hyperparameters.items():
    print(f"  {k}: {v}")

In [ ]:
# Start training
print("Starting K-Means training job...")
print("This will take approximately 3-5 minutes.\n")

kmeans_estimator.fit(
    {'train': train_uri},
    wait=True,
    logs=True
)

In [ ]:
# Get training job info
job_name = kmeans_estimator.latest_training_job.name
print(f"Training job completed: {job_name}")
print(f"Model artifacts: {kmeans_estimator.model_data}")

## Step 5: Deploy and Get Cluster Assignments

In [ ]:
# Deploy the model
print("Deploying K-Means model...")
print("This will take approximately 5-7 minutes.\n")

kmeans_predictor = kmeans_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=f'kmeans-{datetime.now().strftime("%Y%m%d%H%M")}'
)

print(f"\nEndpoint deployed: {kmeans_predictor.endpoint_name}")

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Configure predictor
kmeans_predictor.serializer = CSVSerializer()
kmeans_predictor.deserializer = JSONDeserializer()

def get_cluster_assignments(data, predictor, batch_size=500):
    """
    Get cluster assignments for data.
    
    Returns:
        clusters: Cluster IDs
        distances: Distance to assigned centroid
    """
    clusters = []
    distances = []
    
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        response = predictor.predict(batch)
        
        for pred in response['predictions']:
            clusters.append(pred['closest_cluster'])
            distances.append(pred['distance_to_cluster'])
    
    return np.array(clusters), np.array(distances)

In [ ]:
# Get cluster assignments
print("Getting cluster assignments...")
predicted_clusters, distances = get_cluster_assignments(X, kmeans_predictor)

print(f"\nPredicted cluster distribution:")
unique, counts = np.unique(predicted_clusters, return_counts=True)
for c, n in zip(unique, counts):
    print(f"  Cluster {c}: {n}")

## Step 6: Evaluate Clustering

In [ ]:
# Calculate silhouette score
silhouette = silhouette_score(X, predicted_clusters)

# Calculate inertia (sum of squared distances to centroids)
inertia = np.sum(distances ** 2)

print("Clustering Metrics:")
print("=" * 40)
print(f"Silhouette Score: {silhouette:.4f}")
print(f"Inertia (SSE): {inertia:.2f}")
print(f"Average distance to centroid: {np.mean(distances):.4f}")
print(f"\nSilhouette Score Interpretation:")
print(f"  -1 to 0: Poor clustering")
print(f"  0 to 0.5: Overlapping clusters")
print(f"  0.5 to 1: Dense, well-separated clusters")

In [ ]:
# Visualize predicted clusters
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# True clusters
scatter1 = axes[0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='tab10', alpha=0.6)
axes[0].set_xlabel('Feature 0')
axes[0].set_ylabel('Feature 1')
axes[0].set_title('True Clusters')
plt.colorbar(scatter1, ax=axes[0])

# Predicted clusters
scatter2 = axes[1].scatter(X[:, 0], X[:, 1], c=predicted_clusters, cmap='tab10', alpha=0.6)
axes[1].set_xlabel('Feature 0')
axes[1].set_ylabel('Feature 1')
axes[1].set_title(f'Predicted Clusters (Silhouette: {silhouette:.3f})')
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Distance distribution by cluster
fig, ax = plt.subplots(figsize=(10, 5))

for c in range(NUM_CLUSTERS):
    mask = predicted_clusters == c
    ax.hist(distances[mask], bins=30, alpha=0.5, label=f'Cluster {c}')

ax.set_xlabel('Distance to Centroid')
ax.set_ylabel('Count')
ax.set_title('Distance Distribution by Cluster')
ax.legend()
plt.show()

## Step 7: Choosing k (Elbow Method)

In practice, you would train multiple models with different k values.

In [ ]:
print("""
Choosing the Number of Clusters (k):
====================================

1. Elbow Method:
   - Plot inertia vs k
   - Look for "elbow" where improvement slows

2. Silhouette Method:
   - Plot silhouette score vs k
   - Choose k with highest score

3. Domain Knowledge:
   - Customer segments: Often 3-7
   - Product categories: Based on business needs

4. Gap Statistic:
   - Compare clustering to random reference
   - More statistically rigorous

For production, train multiple models:
- k = 3, 4, 5, 6, 7, 8, 9, 10
- Evaluate each with silhouette score
- Consider business interpretability
""")

## Step 8: Clean Up Resources

In [ ]:
# Delete the endpoint
print(f"Deleting endpoint: {kmeans_predictor.endpoint_name}")
kmeans_predictor.delete_endpoint()
print("Endpoint deleted successfully!")

---

## Summary

In this exercise, you learned:

1. **Data Format**: CSV with features only (no labels)

2. **Key Hyperparameters**:
   - `k`: Number of clusters (most critical)
   - `feature_dim`: Number of features
   - `init_method`: Initialization strategy (use `kmeans++`)
   - `extra_center_factor`: For more stable results
   - `epochs`: Number of passes (1 usually sufficient)

3. **Output**:
   - `closest_cluster`: Assigned cluster ID
   - `distance_to_cluster`: Distance to centroid

4. **Evaluation Metrics**:
   - Silhouette score (-1 to 1, higher is better)
   - Inertia/SSE (lower is better)
   - Mean squared distance to centroid

### Instance Recommendations

| Task | Instance Types | Notes |
|------|----------------|-------|
| Training (small) | ml.m5.large | CPU, <1M samples |
| Training (large) | ml.g4dn.xlarge | GPU, >1M samples |
| Inference | ml.m5.large, ml.c5.large | CPU sufficient |

### Methods to Choose k

| Method | Description | When to Use |
|--------|-------------|-------------|
| **Elbow Method** | Plot inertia vs k, find "elbow" | Quick visual check |
| **Silhouette Score** | Plot score vs k, maximize | More rigorous |
| **Gap Statistic** | Compare to random reference | Statistical approach |
| **Domain Knowledge** | Business understanding | When you know expected groups |

### Silhouette Score Interpretation

| Score Range | Meaning |
|-------------|---------|
| 0.71 - 1.00 | Strong structure |
| 0.51 - 0.70 | Reasonable structure |
| 0.26 - 0.50 | Weak structure, try different k |
| < 0.25 | No substantial structure |

### SageMaker K-Means Advantages

- Handles billions of data points
- Single-pass streaming algorithm
- GPU acceleration
- Distributed training
- Mini-batch for memory efficiency

### Best Practices

1. **Scale features**: K-Means is distance-based, normalize features!
2. **Use kmeans++**: Better initialization = better results
3. **Try multiple k values**: Use elbow/silhouette to find optimal
4. **Increase epochs** if clusters seem unstable
5. **Check cluster sizes**: Very uneven sizes may indicate poor k

### Next Steps

- Try different k values with elbow/silhouette method
- Use cluster assignments as features for downstream models
- Combine with PCA for visualization
- Apply to customer segmentation use case
- Compare with other clustering algorithms (DBSCAN, hierarchical)